# Simulate the striatum example network

This example continues ```striatum_example.ipynb``` and shows how to setup input and run the simulation.

## Setup the input

Here we use the file ```input_config/striatum-test-input.json```.

The input can be generated in serial or in parallel. To generate it in parallel, pass ```rc``` parameter as in the previous examples.

In [1]:
import os
from snudda.input import SnuddaInput

network_path = os.path.join("networks", "striatum_example")

si = SnuddaInput(network_path=network_path, 
                 input_config_file=os.path.join("input_config","striatum-test-input.json"),
                 verbose=False)
si.generate()

Writing spikes to networks/striatum_example/input-spikes.hdf5


## Compile the  NEURON mod files
If you run this on Windows you need to exchange / for \ in the path below.

In [2]:
!nrnivmodl ../../snudda/data/neurons/mechanisms

/home/hjorth/HBP/Snudda/examples/notebooks
ls: cannot access '../../snudda/data/neurons/mechanisms/*.inc': No such file or directory
COBJS=''
"/usr/local/nrn/share/nrn/libtool" --tag=CC --mode=compile mpicc -DHAVE_CONFIG_H  -I. -I.. -I"/usr/local/nrn/include/nrn" -I"/usr/local/nrn/x86_64/lib"      -g -O2 -c -o mod_func.lo mod_func.c
libtool: compile:  mpicc -DHAVE_CONFIG_H -I. -I.. -I/usr/local/nrn/include/nrn -I/usr/local/nrn/x86_64/lib -g -O2 -c mod_func.c  -fPIC -DPIC -o .libs/mod_func.o
"/usr/local/nrn/share/nrn/libtool" --tag=CC --mode=link mpicc -module  -g -O2    -o libnrnmech.la -rpath "/usr/local/nrn/x86_64/lib"  bk_ch.lo bk_fs.lo bk_ms.lo ca_ch.lo cadyn_fs.lo cadyn_ms.lo cal12_ms.lo cal13_ms.lo cal_ch.lo caldyn_ms.lo can_fs.lo can_ms.lo cap_ch.lo caq_fs.lo caq_ms.lo car_fs.lo car_ms.lo cat32_ms.lo cat33_ms.lo concACh.lo concDAfile.lo concDA.lo hcn12_ch.lo hd_lts.lo im_lts.lo it_lts.lo kaf_fs.lo kaf_ms.lo kas_fs.lo kas_ms.lo kcnq_ch.lo kdrbca1_lts.lo kdr_fs.lo kdr_lts.lo kdr_m

## Run the neuron simulation

The below code runs the simulation in serial, which is quite slow. To instead run it in parallel for 2s of simulation you can type:
```mpiexec -n 6 snudda simulate networks/striatum_example --time 2```

In [ ]:
!mpiexec -n 6 snudda simulate networks/striatum_example --time 3.5

numprocs=6
Using input file networks/striatum_example/input-spikes.hdf5
args: Namespace(action='simulate', disable_gj=False, input_file=None, mech_dir=None, network_file=None, path='networks/striatum_example', profile=False, randomseed=None, spikes_out=None, time=3.5, verbose=False, volt_out=None)
Using input file networks/striatum_example/input-spikes.hdf5
args: Namespace(action='simulate', disable_gj=False, input_file=None, mech_dir=None, network_file=None, path='networks/striatum_example', profile=False, randomseed=None, spikes_out=None, time=3.5, verbose=False, volt_out=None)
Using input file networks/striatum_example/input-spikes.hdf5
args: Namespace(action='simulate', disable_gj=False, input_file=None, mech_dir=None, network_file=None, path='networks/striatum_example', profile=False, randomseed=None, spikes_out=None, time=3.5, verbose=False, volt_out=None)
Using input file networks/striatum_example/input-spikes.hdf5
args: Namespace(action='simulate', disable_gj=False, input_file=

See ```striatum_example_plot.ipynb``` for information on plotting a spike raster.